In [152]:
import bz2
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import re
import numpy as np

filename = r'C:\\work\\jul\\python_libs\\ubercorpus.tokenized.shuffled.txt.bz2'
re_alphabet = re.compile(r'[^АаБбВвГгҐґДдЕеЄєЖжЗзИиІіЇїЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЬьЮюЯя\'’]', re.IGNORECASE)
re_spaces = re.compile(r' {2,}')
re_quotes = re.compile(r'\'{2}')


def preprocessor(text):
    s = ' ' + re_alphabet.sub(' ', text) + ' '
    s = s.replace('’', '\'')
    s = re_quotes.sub('', s)
    return re_spaces.sub(' ', s)
    
s = '''Серед інших прикладів – серія Czytam Sobie польського видавництва Egmont Polska, яка виходить у межах промоції самостійного читання.
Серія пропонує три рівні читання і не прив’язана до освітньої си'стеми.
Хоча серед безкоштовних додатків до серії є, зокрема, розробки ''уроків'' та конкурсів за книжками, а також робочі аркуші, відзнаки, плакати тощо, які можуть знадобитися на уроках.
Саме з європейського досвіду і з’явилася ідея серії "Читальні".'''

print(preprocessor(s))
counter_ngram = CountVectorizer(lowercase=False, analyzer='char_wb', ngram_range=(2, 7), min_df=5, max_features = 7000)
analyzer = counter_ngram.build_analyzer()
print(analyzer(s))

 Серед інших прикладів серія польського видавництва яка виходить у межах промоції самостійного читання Серія пропонує три рівні читання і не прив'язана до освітньої си'стеми Хоча серед безкоштовних додатків до серії є зокрема розробки уроків та конкурсів за книжками а також робочі аркуші відзнаки плакати тощо які можуть знадобитися на уроках Саме з європейського досвіду і з'явилася ідея серії Читальні 
[' С', 'Се', 'ер', 'ре', 'ед', 'д ', ' Се', 'Сер', 'ере', 'ред', 'ед ', ' Сер', 'Сере', 'еред', 'ред ', ' Сере', 'Серед', 'еред ', ' Серед', 'Серед ', ' Серед ', ' і', 'ін', 'нш', 'ши', 'их', 'х ', ' ін', 'інш', 'нши', 'ших', 'их ', ' інш', 'інши', 'нших', 'ших ', ' інши', 'інших', 'нших ', ' інших', 'інших ', ' інших ', ' п', 'пр', 'ри', 'ик', 'кл', 'ла', 'ад', 'ді', 'ів', 'в ', ' пр', 'при', 'рик', 'икл', 'кла', 'лад', 'аді', 'дів', 'ів ', ' при', 'прик', 'рикл', 'икла', 'клад', 'ладі', 'адів', 'дів ', ' прик', 'прикл', 'рикла', 'иклад', 'кладі', 'ладів', 'адів ', ' прикл', 'прикла', '

In [154]:
N = 200000
i = 0
counter = Counter()
counter_lower = Counter()

with bz2.BZ2File(filename, 'r') as bz2file:
    line = bz2file.readline()
    while line and i < N:
        s = preprocessor(line.decode('utf-8'))
#         print(s)
        tokens = s.split()
        alpha = [x for x in tokens if x.isalpha()]
        for x in alpha:
            counter[x] += 1
            counter_lower[x.lower()] += 1
        i += 1
        line = bz2file.readline()
print(len(counter), len(counter_lower))

249556 222472


In [158]:
min_df_lower = 3
min_df_upper = 2
counter_upper = Counter()
for x in counter:
    if x[0].isupper():
        xl = x.lower()
        c = counter[x]
        cl = counter[xl]
        cll = counter_lower[xl]
        if cl < 0.5 * cll:
            counter_lower[xl] = cll - c
        if counter_lower[xl] < min_df_lower:
            del counter_lower[xl]            
#             counter_lower[x] = c
        if c >= min_df_upper:
            counter_upper[x] = c

In [159]:
print(len(counter_lower), len(counter_upper))

148648 32824


In [160]:
v = [x for x in counter_lower.most_common(7000) if len(x[0])<5]
print(len(v))
print(v)

831
[('в', 53433), ('на', 51782), ('у', 48171), ('і', 43791), ('з', 38131), ('що', 30183), ('не', 25324), ('до', 22785), ('за', 21755), ('та', 19707), ('про', 14658), ('а', 12610), ('як', 12297), ('це', 11215), ('для', 10404), ('від', 9553), ('року', 9005), ('він', 8228), ('його', 7436), ('й', 7061), ('які', 5855), ('є', 5789), ('із', 5513), ('але', 5420), ('під', 5058), ('було', 4961), ('я', 4926), ('по', 4764), ('ми', 4446), ('час', 4409), ('який', 4023), ('тому', 3891), ('році', 3732), ('був', 3693), ('м', 3673), ('буде', 3600), ('так', 3553), ('того', 3532), ('ще', 3484), ('її', 3348), ('вже', 3291), ('вони', 3252), ('або', 3159), ('щоб', 3116), ('чи', 3102), ('при', 3092), ('може', 2777), ('с', 2746), ('вона', 2740), ('р', 2715), ('коли', 2676), ('де', 2654), ('те', 2626), ('якщо', 2619), ('були', 2597), ('їх', 2593), ('має', 2590), ('ж', 2505), ('щодо', 2477), ('яка', 2470), ('лише', 2314), ('то', 2276), ('була', 2263), ('зі', 2226), ('б', 2169), ('все', 2070), ('прес', 2050), ('

In [161]:
words_dict_uk = []
with open(r'C:\work\jul\python_libs\dict_uk\out\words.txt', 'r', encoding='utf-8') as f:
    line = f.readline()
    while line:
        words_dict_uk.append(line.strip()) 
        line = f.readline()
print(len(words_dict_uk))


3024113


In [162]:
words = list(counter_lower.keys()) + list(counter_upper.keys()) + words_dict_uk
X = counter_ngram.fit_transform(words)
feats = counter_ngram.get_feature_names()
counts = np.asarray(X.sum(axis=0))[0]
print(len(feats), counts.shape)

7000 (7000,)


In [163]:
for w, c in sorted(zip(feats, counts), key = lambda t: t[1]):
    print(w, c)

дод 2484
оцен 2484
льг 2485
нити 2485
татим 2485
ець  2486
круч 2486
ницям 2486
 Ве 2487
вували 2487
анал 2488
ичне 2488
су  2488
еоп 2489
нуватим 2489
ратив 2489
тивніш 2489
трив 2490
 чор 2491
лож 2492
викр 2493
обор 2493
раю 2493
удів 2493
ідпр 2494
дет 2495
ливост 2495
чар 2495
іональн 2495
бас 2496
гото 2496
цяти 2496
іональ 2496
Со 2498
ам'я 2498
вну  2498
товх 2498
штовх 2498
Нов 2499
взаємо 2499
рім 2499
анч 2500
кового  2500
прова 2500
кового 2501
найви 2501
нест 2502
рего 2502
фіт 2502
Ми 2503
аліза 2503
анські 2503
гічні 2503
кнайм 2503
лять 2503
шнь 2503
шньо 2503
якнайм 2503
дерн 2504
ократ 2504
осер 2504
озам 2505
рони 2505
езк 2506
овал 2506
льш 2507
рибу 2507
цем  2507
До 2508
ливос 2508
оща 2508
рент 2508
еол 2509
жлив 2509
зич 2509
стре 2509
тном 2509
 перез 2511
ксу 2512
прові 2512
колі 2513
 агр 2514
кової 2514
кової  2514
рух 2514
Де 2515
цькою 2515
цькою  2515
а- 2516
ковій 2516
ам' 2518
цькій 2518
цькій  2518
жли 2519
инець 2519
инец 2520
люйм 2520
люймо 2520
рну

нице 3020
ов' 3020
спи 3020
дкр 3021
авр 3022
арни 3022
енням  3022
никам  3022
ненсь 3023
ненськ 3023
тель 3023
ючим  3023
вню 3024
ненс 3024
омен 3024
рму 3024
 проб 3025
римі 3025
Бе 3026
мову 3026
онц 3027
розпо 3027
тріл 3027
 пош 3028
зеле 3028
мован 3028
пні 3029
зати 3030
міно 3030
рств 3031
-в 3033
ваност 3033
хло 3033
дає 3034
имся 3034
имся  3034
маж 3034
ьовув 3034
ьовува 3034
 перео 3035
мовн 3035
нізо 3035
 лу 3036
ао 3037
итесь 3037
итесь  3037
проф 3037
нчу 3039
сем 3039
 відт 3040
ючих  3040
езе 3041
итеся 3041
итеся  3041
ную 3041
ільс 3041
запл 3042
шуватим 3042
підр 3043
р' 3043
з' 3045
тих  3045
оліс 3046
 кат 3047
дур 3047
имось 3047
имось  3047
ляк 3047
танов 3047
раво 3048
сувати 3048
ючих 3048
 нек 3049
озпо 3049
 заго 3051
ишс 3051
ишся 3051
ишся  3051
леним 3051
 Ро 3054
дум 3055
лоз 3055
льч 3055
овик 3056
мля 3057
тепл 3057
неро 3058
изован 3059
крат 3059
бд 3060
обуд 3060
арто 3061
аціона 3061
зве 3062
айдо 3064
лиз 3064
онно 3064
лювано 3065
ісі 3065
имос

іну 3742
 найп 3743
чени 3743
шив 3744
 пів 3746
 тран 3747
чил 3748
блюв 3749
 інте 3753
юют 3753
юють 3753
нецьк 3754
альнім  3755
нськім 3755
нськім  3755
доз 3758
опов 3758
овував 3760
овб 3764
огол 3766
альнім 3767
відч 3767
жд 3767
вальним 3769
еку 3771
омлю 3772
виз 3773
зш 3773
перер 3775
ансько 3778
шан 3782
нської  3785
 підп 3786
авле 3786
нської 3786
іот 3787
 проти 3790
ована  3790
остям 3794
знав 3795
тим  3795
шувати 3795
роди 3796
 доб 3797
енов 3797
рика 3797
бло 3798
жени 3798
ованих 3798
ованих  3798
Са 3799
ичк 3799
Бо 3800
ерш 3800
лізова 3804
лізован 3804
цьо 3804
б' 3805
мам 3805
нику 3805
аючо 3807
еса 3807
тере 3807
аши 3809
окра 3813
альною  3814
дост 3814
тало 3817
ела 3818
оше 3821
чуй 3821
 ні 3822
ічніш 3822
альній 3823
альній  3823
ено  3823
ована 3823
Го 3824
нським  3824
омб 3824
нська  3825
пак 3826
цу 3826
 ча 3828
агл 3830
альними 3831
нго 3832
пош 3833
 неза 3834
чою 3834
чою  3834
ярн 3834
ідни 3835
пет 3838
 Ку 3839
во  3839
 фе 3840
агально 3841


ичі 4857
ківсько 4858
нік 4859
аєм  4860
рз 4860
бив 4861
уме 4862
итн 4863
о-п 4863
сек 4864
бат 4865
хм 4866
уванню 4870
уванню  4870
чну 4870
 пра 4872
 ли 4874
ище 4874
озна 4874
тою 4879
 висо 4880
поч 4882
нтер 4883
олоч 4883
чном 4883
чним  4885
юєте 4886
дот 4889
епро 4889
тли 4890
юйт 4890
юйте 4890
 зах 4892
верт 4892
етри 4893
в'яз 4894
чувал 4894
длив 4896
оїд 4900
нько 4901
 обл 4903
иком  4903
емн 4904
нути 4904
тве 4904
юємо 4906
 пи 4909
жую 4909
чна  4916
овіш 4917
ело 4922
сим 4922
так 4922
уючо 4922
чної 4922
чної  4922
чна 4923
ваю 4924
 відк 4926
 гол 4926
іщ 4927
коз 4928
гідро 4930
мод 4932
бел 4933
підп 4935
 зр 4940
 зі 4940
лізов 4940
енко 4941
ніл 4944
енім  4947
уванні 4948
уванні  4948
атов 4951
тає 4952
джа 4954
лял 4955
нз 4955
повн 4955
гну 4956
ювало 4957
кнайс 4958
якнайс 4958
він 4960
город 4960
проб 4960
 нам 4965
фон 4967
ленн 4969
ожн 4969
упл 4969
віз 4971
абу 4974
чних 4975
чних  4975
елик 4977
ерб 4978
ифік 4981
зор 4982
 прим 4984
дне 4985
тово

анц 6689
бно 6690
дч 6692
дем 6695
ванім 6696
ванім  6696
еля 6715
оря 6716
 фі 6717
вост 6722
графі 6727
гри 6730
дст 6736
вець 6743
жні 6743
 вит 6745
ватиму  6750
рти 6750
арб 6751
еме 6753
 зад 6758
ваній  6760
найд 6761
ваній 6764
піл 6769
рус 6771
овий  6783
бла 6785
ідг 6785
ваною 6786
ваною  6786
прог 6786
івко 6786
ваному  6788
остя 6796
ванню 6797
ванню  6797
лоч 6799
пус 6801
ваному 6802
мок 6803
ваними 6811
ваними  6811
 ук 6815
оско 6825
рст 6825
вец 6828
конт 6835
ійськ 6835
ида 6837
 зал 6841
лід 6849
 одн 6851
ваним  6852
ваної  6860
 под 6866
-п 6867
нічн 6867
рц 6868
лит 6869
оліт 6870
олов 6871
радіо 6871
стк 6871
тично 6871
ваної 6874
фл 6875
ометр 6877
ахо 6881
вану  6883
ано  6884
ованіш 6885
ема 6886
овий 6886
ване  6887
блю 6889
пло 6890
зуватим 6892
вану 6899
ватиме  6899
ваного 6902
ваного  6902
змо 6905
опу 6907
ванні  6909
нальн 6912
ваног 6920
ваном 6922
рош 6926
ідв 6931
вб 6932
рта 6942
 бл 6943
ванням  6943
зьк 6949
ідт 6954
мув 6960
мува 6960
 гл 6961
к

ача 10014
льніш 10018
оми 10018
ьніш 10018
ту  10020
дув 10023
лектр 10031
 зат 10033
оне 10041
 авт 10043
вавши  10044
ноп 10047
инн 10052
бс 10053
риг 10056
ерм 10068
прот 10072
 си 10073
пат 10077
обу 10079
рост 10079
оду 10086
ощ 10086
 Р 10088
ючі 10088
вує 10093
гар 10096
ок  10097
дко 10114
іні 10120
ану  10124
ється  10135
єтьс 10136
ється 10136
єть 10137
влю 10170
чим 10175
уся 10177
рх 10180
 Т 10184
ана  10190
ода 10206
аї 10207
ітн 10213
їв 10215
осо 10216
вало  10217
ини 10226
бов 10259
еа 10259
лено 10261
ішій  10262
ішій 10264
ічні 10268
 фо 10275
іза 10280
ішої 10281
ішої  10281
цю 10287
ішім  10287
айд 10292
ішому  10293
ішом 10294
ішому 10294
 зн 10298
пост 10316
авсь 10324
ішого  10324
ішого 10325
 вип 10327
зар 10332
ішім 10334
ішою 10335
ішою  10335
анк 10338
ньо 10338
ішими  10340
ішими 10345
ішог 10349
іша  10353
ючи  10359
ійно 10366
ання  10382
ішу  10383
іва 10389
іший  10392
еним 10394
іший 10394
іших  10395
іших 10397
бал 10409
ішим  10411
 розп 10413
азо 10

чува 20870
ина 20891
рот 20904
но- 20943
чув 20944
еро 20956
рит 20966
уєт 20978
ням 21003
мін 21021
том 21033
поп 21072
мен 21081
ове 21104
 те 21136
зов 21136
ватимет 21160
нап 21163
ніші 21187
оти 21250
тан 21373
пів 21389
мн 21401
віш 21404
мут 21436
кри 21498
зе 21599
над 21656
джу 21698
нал 21702
фа 21712
цій 21716
еб 21732
ана 21751
дв 21764
жен 21834
ліз 21880
оо 21895
 тр 21912
нц 21920
вис 21926
чк 21952
ара 21963
вськи 22030
св 22062
оза 22075
тал 22099
ем  22123
тру 22177
ког 22181
шим 22243
ман 22270
рд 22275
тив 22412
ті  22418
 ре 22461
ніст 22473
чни 22484
иб 22537
альни 22585
мся 22650
мся  22650
ага 22653
юватим 22778
ино 22799
чен 22868
уть 22919
ця 22928
анс 23065
жува 23140
жув 23146
ниц 23203
ей 23230
рат 23230
 С 23237
нен 23249
яти 23252
ки  23298
зд 23422
ец 23471
ювал 23476
ька 23477
уванн 23666
вало 23672
всько 23709
вала 23730
 ч 23762
авши 23769
лог 23830
нів 23874
кол 23901
мов 23904
амо 24031
ект 24034
аф 24055
пос 24060
иві 24077
авш 24081
жн 24129
рен 2

In [164]:
z = counter_ngram.transform(['"Забудь її у лісі" - заявив Микола.'])
print(counter_ngram.inverse_transform(z))

[array([' М', ' з', ' за', ' л', ' лі', ' у', 'За', 'Ми', 'аб', 'абу',
       'ая', 'бу', 'буд', 'в ', 'ви', 'вив', 'дь', 'за', 'ив', 'ив ',
       'ик', 'ико', 'икол', 'ко', 'кол', 'ла', 'лі', 'ліс', 'ол', 'ола',
       'сі', 'у ', 'уд', 'ь ', 'яв', 'іс', 'ісі', 'ї '], dtype='<U7')]


In [167]:
vocabulary_keys = feats + [' ' + w + ' ' for w in counter_lower]
# for w in counter_lower:
#     s = ' '+ w +' '
#     if s not in vocabulary:
#         vocabulary[s] = len(vocabulary)
# print(len(vocabulary))
counter_ngram.set_params(vocabulary=vocabulary_keys)
print(len(counter_ngram.vocabulary_))
print(len(counter_ngram.get_params()))



# z = counter_ngram.transform(['"Забудь її у лісі" - заявив Микола.'])
z = counter_ngram.transform(['м\'ясо'])
print(counter_ngram.inverse_transform(z))

7000
17
[array([' м', "'я", "м'", "м'я", 'о ', 'со', 'яс'], dtype='<U7')]


In [168]:
from sklearn.externals import joblib
joblib.dump(counter_ngram.get_params(), 'ngram_vectorizer_params.pkl') 

['ngram_vectorizer_params.pkl']